<a href="https://colab.research.google.com/github/ragul-n/Gender-bias-in-Indian-cinema/blob/master/word2vec/word2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysrt
!pip install gensim


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 104 kB 7.2 MB/s 
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=a0d07e4f8e5fac83cf4888793815ab4f7420ad63078c21022a65e5d03682d1b7
  Stored in directory: /root/.cache/pip/wheels/30/a6/ab/4705174e11f44e74d58c14b32edbacbc852644f86658316aef
Successfully built pysrt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np 
import pysrt
import os 
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim 
from gensim.parsing.preprocessing import preprocess_documents,preprocess_string
import tqdm
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.tree import Tree
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_multiple_whitespaces,strip_numeric


def remove_named_entities(tokens):
    # Part-of-speech tag the tokens
    pos_tags = nltk.pos_tag(tokens)
    
    # Identify named entities
    named_entities = nltk.ne_chunk(pos_tags)
    filtered_tokens = [leaf[0] for leaf in named_entities if type(leaf) != nltk.Tree]
    
    return filtered_tokens


class SentenceGenerator():
    def __init__(self, dirname):
        self.stop_words = nltk.corpus.stopwords.words('english')

        self.dirname = dirname

        self.subtitles_list=[]
        self.sentences=[]


        for filename in os.listdir(self.dirname):
            # Get the full path of the file
            filepath = os.path.join(self.dirname, filename)
            
            # Check the size of the file
            if os.path.getsize(filepath) > 1024*15:
                # If the file is larger than the specified size, add it to the list
                self.subtitles_list.append(filepath)
 
    def __iter__(self):
        for fpath in tqdm.tqdm(self.subtitles_list):

            text=pysrt.open(fpath).text.lower()
            text= text.replace("..."," ")
            text= text.replace("\n",". ")

            for sent in sent_tokenize(text):
                sent=preprocess_string(sent, filters=[strip_tags, strip_punctuation, strip_multiple_whitespaces,strip_numeric])
                sent=remove_named_entities(sent)
                #sent=gensim.parsing.preprocessing.remove_stopword_tokens.remove_stopword_tokens(sent)
                sent=[token for token in sent if token not in self.stop_words]
                
                if len(sent)<2:
                    continue
                yield sent

sentences=SentenceGenerator("/content/drive/MyDrive/Dataset/subtitles")

In [ ]:
with open('/content/drive/MyDrive/Dataset/subtitles_sents.txt', 'w') as f:
    for sent in sentences:
        f.write(' '.join(sent) + '\n')

100%|██████████| 4168/4168 [4:19:00<00:00,  3.73s/it]


In [ ]:
from nltk.stem.snowball import SnowballStemmer

def read_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    sents=[line.split() for line in lines]

    stemmer = SnowballStemmer("english")

    stemed_sents=[[stemmer.stem(word) for word in sent] for sent in sents]
    return  stemed_sents

In [ ]:
sents = read_file("/content/drive/MyDrive/Dataset/subtitles_sents.txt")

4512730

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

        model.save('/content/drive/MyDrive/Dataset/word2vec/subtitles_v6')
        model.wv.save("/content/drive/MyDrive/Dataset/word2vec/subtitles_word_emb_v6.word_vectors")





monitor = callback()

model = gensim.models.Word2Vec(sents, size=200, window=5, min_count=25, workers=2,
                               sg=1,negative=5, compute_loss=True,
                               iter = 10, callbacks=[monitor])


Loss after epoch 0: 35369760.0
Loss after epoch 1: 20944184.0
Loss after epoch 2: 11133464.0
Loss after epoch 3: 696832.0
Loss after epoch 4: 679648.0
Loss after epoch 5: 651848.0
Loss after epoch 6: 627240.0
Loss after epoch 7: 586824.0
Loss after epoch 8: 535392.0
Loss after epoch 9: 500328.0
